In [7]:
from torchvision import models, transforms
from torchsummary import summary
import torch
import numpy as np
import cv2
import PIL
import matplotlib.pyplot as plt

import sys
import os

current_dir = os.getcwd()
sys.path.append(os.path.dirname(current_dir))

from lib.src import *
from lib.utils import *

from pytorch_grad_cam import GradCAM, HiResCAM, AblationCAM, ScoreCAM, GradCAMPlusPlus, XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

In [8]:
id_name = dict()
dups_id_name = dict()

for i, name in enumerate(open('imagenet_classes.txt', 'r')):
    name = name[:-1]
    name = name.lower()
    name = name.replace(' ', '_')
    id_name[i] = name

len(id_name)
hash_name = dict()
dups_hash_name = dict()

for line in open('imagenet.txt', 'r'):
    line = line[:-1]
    h, i, n = line.split()
    n = n.lower()
    hash_name[h] = n
len(hash_name)
DICT = dict()

for k1, v1 in id_name.items():
    for k2, v2 in hash_name.items():
        if v1 == v2:
            DICT[k1] = (k1, k2, v1)
            DICT[k2] = (k1, k2, v1)
len(DICT)

2000

In [11]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.01
CLASSES = 20
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

100%|██████████| 20/20 [01:07<00:00,  3.40s/it]


In [12]:
before_arr = np.array(before)
after_arr = np.array(after)
np.maximum(0, -before_arr + after_arr).sum() / before_arr.shape[0], (before_arr < after_arr).sum() / before_arr.shape[0]

(0.0056558677020348115, 0.4)